# Accessing CMR Cloud Collections 

References:
* https://github.com/nsidc/earthdata/blob/main/notebooks/Demo.ipynb
* https://nasa-openscapes.github.io/earthdata-cloud-cookbook/examples/NSIDC/ICESat2-CMR-AWS-S3.html
* https://nasa-openscapes.github.io/2021-Cloud-Workshop-AGU/how-tos/Earthdata_Cloud__Single_File__Direct_S3_Access_NetCDF4_Example.html

Programmatic access of NSIDC data can happen in 2 ways:

```text
Search -> Download -> Process -> Research
```

<img src="https://raw.githubusercontent.com/NASA-Openscapes/earthdata-cloud-cookbook/main/examples/NSIDC/img/download-model.png" width="35%"/>

```text
Search -> Process in the cloud -> Research
```

<img src="https://raw.githubusercontent.com/NASA-Openscapes/earthdata-cloud-cookbook/main/examples/NSIDC/img/cloud-model.png" width="35%"/>

> **Credit**: Open Architecture for scalable cloud-based data analytics. From Abernathey, Ryan (2020): Data Access Modes in Science.


## The big picture: 

There is nothing wrong with downloading data to our local machine but that can get complicated or even impossible if a dataset is too large.
For this reason NSIDC along with other NASA data centers started to collocate or migrate their dataset holdings to the cloud. 

### Steps

1. Authenticate with the NASA Earthdata Login API (EDL).
2. Search granules/collections using a CMR client that supports authentication
3. Parse CMR responses and get AWS S3 URLs
4. Access the data granules using temporary AWS credentials given by the NSIDC cloud credentials endpoint

### Data used:

*  ICESat-2 [ATL03](https://nsidc.org/data/ATL03/versions/4): This data set contains height above the WGS 84 ellipsoid (ITRF2014 reference frame), latitude, longitude, and time for all photons.

### Requirements

* [NASA Eartdata Login (EDL) credentials](https://urs.earthdata.nasa.gov/)
* python libraries:
  - h5py
  - matplotlib
  - xarray
  - s3fs
  - https://github.com/nsidc/earthdata/tree/main
  
  
Another python tutorial for finding data with the python requests
* https://nasa-openscapes.github.io/2021-Cloud-Hackathon/tutorials/01_Data_Discovery_CMR.html
* https://github.com/nasa/eo-metadata-tools/tree/master/CMR/python

In [ ]:
# !pip install earthdata
# !pip install 's3fs<2022.0.0,>=2021.8.1'

## Querying CMR for NSIDC data in the cloud

Most collections at NSIDC have not being migrated to the cloud and can be found using CMR with no authentication at all. Here is a simple example for 
altimeter data (ATL03) coming from the ICESat-2 mission. First we'll search the regular collection and then we'll do the same using the cloud collection.

**Note**: This notebook uses CMR to search and locate the data granules, this is not the only workflow for data access and discovery. 

* **HarmonyPy**: Uses **Harmony** the NASA API to search, subset and transform the data in the cloud.
* **cmr-stac**: A "static" metadata catalog than can be read by **Intake** oand other client libraries to optimize the access of files in the cloud.



## Cloud Collections

Some NSIDC cloud collections are not yet, which means that temporarily you'll have to request access emailing nsidc@nsidc.org so your Eartdata login is in the authorized list for early users.

In [1]:
from earthdata import Auth, DataGranules, DataCollections, Accessor

auth = Auth() # if we want to access NASA DATA in the cloud
auth.login()

Enter your Earthdata Login username:  aimeeb
Enter your Earthdata password:  ··················


You're now authenticated with NASA Earthdata Login


True

In [3]:
auth._session

In [ ]:
collections = DataCollections(auth).keyword('ICESat-2').get(10)
collections[0]['meta']

In [ ]:
granules = DataGranules(auth).short_name('ATL08').bounding_box(-10,20,10,50).get(5)
[display(g) for g in granules[0:5]]

## Data Access using AWS S3

* **IMPORTANT**: This section will only work if this notebook is running on the AWS **us-west-2** zone

There is more than one way of accessing data on AWS S3, either downloading it to your local machine using the official client library or using a python library. 

**Performance tip**: using the HTTPS URLs will decrease the access performance since these links have to internally be processed by AWS's content delivery system (CloudFront). To get a better performance we should access the `S3://` URLs with BOTO3 or a high level S3 enabled library (i.e. S3FS)


Related links:
* [HDF in the Cloud challenges and solutions for scientific data](http://matthewrocklin.com/blog/work/2018/02/06/hdf-in-the-cloud)
* [Cloud Storage (Amazon S3) HDF5 Connector](https://www.hdfgroup.org/solutions/enterprise-support/cloud-amazon-s3-storage-hdf5-connector/)


In [ ]:
import s3fs
import h5py

In [ ]:
access = Accessor(auth)

In [ ]:
Query = DataGranules().concept_id("C2144424132-NSIDC_ECS")

In [ ]:
Query.hits()

In [ ]:
granules = Query.get(100)

In [ ]:
granules[0].cloud_hosted

In [ ]:
import os
import glob

atl08_dir = 'data/demo-atl08'
files = glob.glob(f'{atl08_dir}/*.h5')

for f in files:
    try:
        os.remove(f)
    except OSError as e:
        print("Error: %s : %s" % (f, e.strerror))

In [ ]:
%%time
files = access.get(granules[0:3], atl08_dir)

In [ ]:
import h5py

file = 'data/demo-atl08/ATL08_20181014001920_02350103_005_01.h5'
ds = h5py.File(file, 'r')
ds

# Cloud-Hosted Granules

In [ ]:
Query = DataGranules().concept_id("C1968980609-POCLOUD").bounding_box(-134.7,54.9,-100.9,69.2)
print(f"Granule hits: {Query.hits()}")
cloud_granules = Query.get(100)
# is this a cloud hosted data granule?
cloud_granules[0].cloud_hosted

## Using xarray to open files on S3

ATL data is complex so xarray doesn't know how to extract the important bits out of it.

In [ ]:
# Let's order them by size again.
import operator
cloud_granules_by_size = sorted(cloud_granules, key=operator.itemgetter("size"))
# now our array is sorted by size from less to more. Let's print the first 10
cloud_granules_by_size[0:2]

In [ ]:
%%time

jason_dir = 'data/jason'
#files = access.get(cloud_granules_by_size[0:2], jason_dir)

In [ ]:
s3_cred_endpoint = {
    'podaac':'https://archive.podaac.earthdata.nasa.gov/s3credentials',
    'gesdisc': 'https://data.gesdisc.earthdata.nasa.gov/s3credentials',
    'lpdaac':'https://data.lpdaac.earthdatacloud.nasa.gov/s3credentials',
    'ornldaac': 'https://data.ornldaac.earthdata.nasa.gov/s3credentials',
    'ghrcdaac': 'https://data.ghrc.earthdata.nasa.gov/s3credentials'
}

In [ ]:
def get_temp_creds(daac):
    temp_creds_url = s3_cred_endpoint
    return requests.get(temp_creds_url[daac]).json()

In [ ]:
import requests
temp_creds_req = get_temp_creds('podaac')

In [ ]:
#!pip install boto3
import os
import requests 
import boto3
from osgeo import gdal
import rasterio as rio
from rasterio.session import AWSSession
import rioxarray
import hvplot.xarray
import holoviews as hv

In [ ]:
fs_s3 = s3fs.S3FileSystem(anon=False, 
                          key=temp_creds_req['accessKeyId'], 
                          secret=temp_creds_req['secretAccessKey'], 
                          token=temp_creds_req['sessionToken'])

In [ ]:
#s3_url = cloud_granules[0].data_links()[0]
s3_url = 's3://podaac-ops-cumulus-protected/ECCO_L4_SSH_05DEG_MONTHLY_V4R4/SEA_SURFACE_HEIGHT_mon_mean_2015-01_ECCO_V4r4_latlon_0p50deg.nc'

In [ ]:
s3_file_obj = fs_s3.open(s3_url, mode='rb')

In [ ]:
s3_file_obj

In [ ]:
ssh_ds = xr.open_dataset(s3_file_obj, engine='h5netcdf')
ssh_ds

In [ ]:
ssh_da = ssh_ds.SSH
ssh_da

In [ ]:
ssh_da.hvplot.image(x='longitude', y='latitude', cmap='Spectral_r', aspect='equal').opts(clim=(ssh_da.attrs['valid_min'],ssh_da.attrs['valid_max']))